### Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
#Sklearn
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE

#other
import concurrent.futures
import time
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
warnings.filterwarnings('ignore')


%matplotlib inline

# Plotly 
from plotly import tools
import chart_studio.plotly as py
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.figure_factory as ff

#spaCy
import spacy
nlp = spacy.load("fr_core_news_lg")
from spacy.lang.fr.stop_words import STOP_WORDS
from spacy.lang.fr import French
import string
punctuations = string.punctuation
stopwords = list(STOP_WORDS)

### Load (acceptably clean) data

In [2]:
data=pd.read_pickle("../../data/Text_clean0.pkl")
data_np=data.to_numpy()

In [6]:
data_np[251]

"31 Station météo Mons station épuration. Toulouse métropole. le texte suivant est a utiliser pour décrire le jeu de données:Ce jeu de données est issu du capteur n°31 situé au pied de la commune de Mons qui fait parti d'un réseau de stations météo situées sur la métropole Toulousaine en vue d'étudier le phénomène d'Ilot de Chaleur Urbain (ICU). environnement,ilot-de-chaleur,meteo,precipitations,station-meteo,temperature"

In [7]:
import progressbar
bar = progressbar.ProgressBar(maxval=len(data_np), \
    widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])


In [8]:
i=random.randint(1,len(data_np))

print(i)

16826


In [9]:
spacy.displacy.render(nlp(data_np[i]), style='ent',jupyter=True)

## Process data to extract keywords only

### Locations remove

In [10]:
LOCS=[]
for i in range(len(data_np[i])):
    for ent in nlp(data_np[i]).ents:
        if ent.label_ == 'LOC':
            LOCS.append(ent.text)

In [11]:
LOCS

['Paris',
 'Laboratoire',
 'Paris',
 'Paris',
 'Cartographie des laboratoires](https://capgeo.maps.arcgis.com',
 'Paris',
 'Alpes',
 'Maritimes',
 'Paris',
 'Ville de Nice',
 'Chambéry',
 'Chambéry',
 "communauté d'agglomération Grand Chambéry",
 'Grand Chambéry',
 'Grand Chambéry',
 'Chambéry',
 "communauté d'agglomération Grand Chambéry",
 'Corse',
 'Corse',
 'Monreseaumobile.fr](http://monreseaumobile.fr/',
 'Corse',
 'Corse',
 'Monreseaumobile.fr](http://monreseaumobile.fr/',
 'Orléans Métropole',
 'Orléans Métropole',
 'Orléans Métropole',
 'Orléans Métropole',
 'Routiers',
 'Saint-Denis',
 'La Réunion',
 'Points Routiers',
 'PR',
 'Saint-Denis',
 'lInstitut Culturel Basque',
 'CAPVERN',
 'Capvern',
 'département des Hautes-Pyrénées',
 'Occitanie',
 'Tronçons',
 'Gamarthe',
 'Gamarthe',
 'lInstitut Culturel Basque',
 'Gamarthe',
 'Gamarthe',
 'Gamarthe',
 'BAL',
 'RGF93',
 'CAPVERN',
 'Île-de-France',
 'Île-de-France',
 'Copie_jardins',
 'Paris Seine Ouest',
 'Territoire de Grand 

### Quicker to remove basics

In [12]:
parser=French()
def spacy_tokenizer(sentence):
    tokens=parser(sentence)
    tokens = [ word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in stopwords and word not in punctuations ]
    tokens = " ".join([i for i in tokens])
    return tokens

In [13]:
spacy_tokenizer(data_np[i])

'boucles randonnée métropole lyon métropole lyon proposition parcours chemins inscrits plan départemental métropolitain itinéraires pédestres randonnées pdmipr \r descriptions itinéraires site départ arrivée intérêt patrimonial \r difficulté \r temps parcours \r dénivelé__origine__sources données grand lyon département rhône__organisations partenaires__métropole lyon__liens annexes evg_esp_veg.envpdiprboucle/all.json](https://download.data.grandlyon.com/ws/grandlyon/evg_esp_veg.envpdiprboucle/all.json consulter fiche geo.data.gouv.fr](https://geo.data.gouv.fr fr datasets/935ef629a353e75d67f26dcc1976736d0c9673d8 activite air donnees ouvertes environment environnement passerelle inspire patrimoine culturel patrimoine paysage sentier sentier randonnee sentier pedestre sport'

In [14]:
data_np_proc0=[]
bar.start()
for i in range(len(data_np)):
    data_np_proc0.append(spacy_tokenizer(data_np[i]))
    bar.update(i+1)
bar.finish()

[========================================================================] 100%


In [18]:
data_np_proc0[i]

'carrefour arles crau camargue montagnette implantation carrefours linéaires voie axe carrefour donnees publiques filaire france localisation nom referentiels coordonnees reseau routier transport voie'

In [16]:
data_proc0_pd=pd.DataFrame(data_np_proc0, dtype="str")

In [17]:
data_proc0_pd.to_csv('data_proc0_pd.pkl')

### Remove the locs (there might be a better way)

In [19]:
from spacy.attrs import LOWER, POS, ENT_TYPE, IS_ALPHA
from spacy.tokens import Doc
def remove_tokens_on_match(doc):
    indexes = []
    for index, token in enumerate(doc):
        if (token.ent_type_ =='LOC') :
            indexes.append(index)
    np_array = doc.to_array([LOWER, POS, ENT_TYPE, IS_ALPHA])
    np_array = np.delete(np_array, indexes, axis = 0)
    doc2 = Doc(doc.vocab, words=[t.text for i, t in enumerate(doc) if i not in indexes])
    doc2.from_array([LOWER, POS, ENT_TYPE, IS_ALPHA], np_array)
    return doc2

In [20]:
data_np_proc1=[]
bar.start()
for i in range(len(data_np_proc0)):
    data_np_proc1.append(remove_tokens_on_match(nlp(data_np_proc0[i])))
    bar.update(i+1)
bar.finish()

In [24]:
data_np_proc1[261].text

'arrêts horaires parcours théoriques gtfs réseau routier régional transport scolaire interurbain 60 \x96 région fichiers gtfs réseau routier régional transport scolaire interurbain 60 composé 3 lots .. circuit scolaire donnee donnees ouvertes flux gtfs hauts france licence odbl ligne reguliere oise point d arret reseaux transport transport transport interurbain transport scolaire '

In [27]:
data_np_proc1_str=list(map(str, data_np_proc1))

In [29]:
np.save('../../data/data_proc1.npy', np.array(data_np_proc1_str), allow_pickle=True)

In [8]:
data_np_proc1_str=np.load('../../data/data_proc1.npy')

In [19]:
data_flat1=''.join(data_np_proc1_str)

In [208]:
data_flat=''.join(str(data_np_proc0))

In [21]:
nlp.max_length=10000000

In [24]:
len(data_flat1)/nlp.max_length

4.2125348

In [22]:
from collections import Counter

# all tokens that arent stop words or punctuations
words = [token.text for token in nlp(data_flat1[:1000000])] 

# five most common tokens
word_freq = Counter(words)
common_words = word_freq.most_common(50)

In [23]:
common_words

[('données', 1642),
 ('..', 659),
 ('   ', 567),
 ('région', 541),
 ('ville', 448),
 ('donnees', 443),
 ('\r  ', 437),
 ('jeu', 421),
 ('nombre', 393),
 ('ouvertes', 381),
 ('département', 348),
 ('réseau', 344),
 ('consulter', 339),
 ('base', 338),
 ('communes', 332),
 ('fr', 323),
 ('2020', 319),
 ('territoire', 309),
 ('eau', 306),
 ('commune', 273),
 ('datasets', 269),
 ('métropole', 267),
 ('nan', 267),
 ('inspire', 267),
 ('zones', 257),
 ('transport', 255),
 ('grand', 252),
 ('fiche', 246),
 ('site', 237),
 ('jour', 233),
 ('code', 229),
 ('geo.data.gouv.fr](https://geo.data.gouv.fr', 228),
 ('passerelle', 227),
 ('franche', 219),
 ('publique', 218),
 ('public', 217),
 ('france', 214),
 ('santé', 213),
 ('points', 213),
 ('2019', 209),
 ('covid-19', 206),
 ('liste', 205),
 ('information', 202),
 ('»', 197),
 ('«', 196),
 ('bourgogne', 196),
 ('informations', 193),
 ('     ', 193),
 ('mise', 192),
 ('territoires', 191)]

In [225]:
print(data_np_proc0[25])
print(data_np[25])

données relatives résultats tests virologiques covid-19 région centre val loire région centre val loire présent jeu données renseigne échelle départementale régionale nombre personnes testées nombre personnes déclarées positives classe âge quotidiennement source   santé publique france mission améliorer protéger santé populations crise sanitaire liée épidémie covid-19 santé publique france charge surveiller comprendre dynamique épidémie anticiper scénarii mettre place actions prévenir limiter transmission virus territoire national système dinformation dépistage dep déploiement 13 mai 2020 plateforme sécurisée systématiquement enregistrés résultats laboratoires tests rt pcr réalisés lensemble laboratoires ville établissements hospitaliers sars cov2.la création système information autorisée durée 6 mois compter fin état urgence sanitaire application décret n° 2020 551 12 mai 2020](https://www.legifrance.gouv.fr affichtexte.do?cidtexte jorftext000041869923 relatif systèmes dinformation me

In [246]:
data_np_proc1_str=list(map(str, data_np_proc1))

### Vectorizer read documentation again

In [25]:
vectorizer = CountVectorizer(min_df=5, max_df=1500, lowercase=True, strip_accents='unicode', ngram_range=(1,2), token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(data_np_proc1_str)

### LDA read documentation and paper again

In [26]:
NUM_TOPICS = 20

In [27]:
# Latent Dirichlet Allocation Model
lda = LatentDirichletAllocation(n_components=NUM_TOPICS, max_iter=10, learning_method='online',verbose=True)
data_lda = lda.fit_transform(data_vectorized)

iteration: 1 of max_iter: 10
iteration: 2 of max_iter: 10
iteration: 3 of max_iter: 10
iteration: 4 of max_iter: 10
iteration: 5 of max_iter: 10
iteration: 6 of max_iter: 10
iteration: 7 of max_iter: 10
iteration: 8 of max_iter: 10
iteration: 9 of max_iter: 10
iteration: 10 of max_iter: 10


In [252]:
np.save('../src/data_ldav2.npy', data_lda, allow_pickle=True)

In [28]:
# Keywords for each topic
def selected_topics(model, vectorizer, top_n=10):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names()[i], topic[i])
                        for i in topic.argsort()[:-top_n - 1:-1]]) 

In [29]:
print("LDA Model:")
selected_topics(lda, vectorizer)

LDA Model:
Topic 0:
[('acte', 1597.2499510945543), ('saint', 1438.0320168861183), ('departements', 1361.4958473528168), ('compter', 1354.176010400212), ('contrainte', 1304.8342719836764), ('mois', 1219.3083300219876), ('subventions', 1187.1077715307256), ('caracteristique', 1131.0206341563944), ('savoie', 1107.7125787999769), ('acte officiel', 1070.4528475203426)]
Topic 1:
[('pays', 2914.154857942029), ('somme', 2136.322585340036), ('surfaces', 1832.1189483024855), ('jeux', 1817.1850758717226), ('aube', 1599.7120124987055), ('pays loire', 1517.5809940829593), ('population', 1503.9581191090915), ('jeux donnees', 1369.6793112203356), ('equipes', 1348.1643632596556), ('supplementaire', 1264.2757463080404)]
Topic 2:
[('nan nan', 3273.7263627961365), ('eaux', 2405.1796557263574), ('deau', 1868.2561475627112), ('bassin', 1567.7407732833694), ('dce', 1487.355679128329), ('raison', 1362.3650896552926), ('true', 1346.5709122307298), ('corse', 1216.4974043783245), ('masses', 1147.6907845335063),

In [30]:
pyLDAvis.enable_notebook()
dash = pyLDAvis.sklearn.prepare(lda, data_vectorized, vectorizer, mds='tsne')
dash

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
9      -10.878068   10.538728       1        1  10.897568
8      -60.884026  -75.591820       2        1   7.780329
15      -2.336119  -53.289074       3        1   7.715404
4      -61.385319  -12.914634       4        1   7.592721
1      -28.737743  126.296997       5        1   5.762096
5        1.066606 -123.552811       6        1   5.249223
7       45.140720   -7.947869       7        1   5.241948
14    -140.795975  -98.530548       8        1   4.580918
2       61.531681  -83.240562       9        1   4.511731
12     -72.563057 -140.396881      10        1   4.481153
0       15.984491   63.578186      11        1   4.473879
18    -160.851624   68.945145      12        1   3.990721
6       85.961830   51.922935      13        1   3.967419
17     -99.020836  109.936348      14        1   3.858064
13    -176.764130   -8.681684      15        1   3.560425
16    -117.189880  -37.420200      16        1   3.475923
10     108.829918  -22.946203      17        1   3.370440
3       47.588097  121.934814      18        1   3.360305
19    -106.467834   30.176889      19        1   3.145967
11     -50.613804   59.328842      20        1   2.983767, topic_info=            Term         Freq        Total Category  logprob  loglift
16231       cnig  3966.000000  3966.000000  Default  30.0000  30.0000
62058    nan nan  3406.000000  3406.000000  Default  29.0000  29.0000
34603  elections  2783.000000  2783.000000  Default  28.0000  28.0000
11475      bruit  3919.000000  3919.000000  Default  27.0000  27.0000
68346       pays  3034.000000  3034.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
21594       cote   899.493333  2526.586823  Topic20  -5.0018   2.4792
36208     entite   648.162613  2044.216807  Topic20  -5.3294   2.3634
25045       deau   582.579930  2526.945416  Topic20  -5.4361   2.0447
89646    surface   508.234848  1690.303334  Topic20  -5.5726   2.3103
90073    systeme   499.499414  1851.412185  Topic20  -5.5900   2.2019

[933 rows x 6 columns], token_table=       Topic      Freq                Term
term                                      
789        3  0.999360               -ldd-
939        1  0.078259  aaaannnn consulter
939       14  0.037839  aaaannnn consulter
939       18  0.883206  aaaannnn consulter
1314       6  0.445821               acces
...      ...       ...                 ...
98748      1  0.999545      zones secteurs
98752      4  0.999146      zones soumises
98772      2  0.966213      zones urbaines
98772     12  0.032928      zones urbaines
98783      4  0.999668         zones zones

[1527 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 9, 16, 5, 2, 6, 8, 15, 3, 13, 1, 19, 7, 18, 14, 17, 11, 4, 20, 12])